***Importing and loading the dataset***

In [1]:
import json
from typing import TypedDict, Optional, Dict, Any
from langgraph.graph import StateGraph

DB_PATH = "D:\Citizens_Rights_Navigator\database\complete_30_database.json"

with open(DB_PATH, "r", encoding="utf-8") as f:
    KB = json.load(f)

print("Database loaded successfully")
print("Total records:", len(KB))

Database loaded successfully
Total records: 27


***Intent Detection***

In [2]:
def detect_intent(query, kb):
    query = query.lower()
    for entry in kb:
        for kw in entry["keywords"]:
            if kw.lower() in query:
                return entry
    return None

***Testing***

In [3]:
entry = detect_intent("बिजली बिल बहुत ज्यादा आया है", KB)
print(entry["title_hi"])

उच्च बिजली बिल विवाद


***Guidance Builder***

In [4]:
def build_guidance(entry, language="English"):
    if not entry:
        return "❌ Sorry, I could not identify your grievance category."

    title = entry["title_hi"] if language == "Hindi" else entry["title"]

    steps = "\n".join([f"{i+1}. {s}" for i, s in enumerate(entry["steps"])])
    documents = "\n".join([f"- {d}" for d in entry["documents"]])
    escalation = "\n".join([f"- {e}" for e in entry["escalation_path"]])
    links = "\n".join(entry["official_links"])
    contacts = "\n".join(entry.get("contact_numbers", []))

    timeline = "\n".join(
        [f"{k.replace('_',' ').title()}: {v}" for k, v in entry["timeline"].items()]
    )

    return f"""
## 🏛 {title}

📄 **Description**  
{entry['description']}

---

### ✅ Step-by-Step Process
{steps}

---

### ⏳ Timelines
{timeline}

---

### 📁 Documents Required
{documents}

---

### 🚨 Escalation Path
{escalation}

---

### 🔗 Official Links
{links}

---

### ☎️ Helpline / Contact
{contacts}

---

⚠️ *This is NOT legal advice. This information is for citizen guidance only.*
"""

***Testing***

In [5]:
entry = detect_intent("cyber fraud of 5000 rupees", KB)
print(build_guidance(entry, "English"))


## 🏛 Online Fraud / Cybercrime Complaint

📄 **Description**  
Report online fraud, UPI scam, phishing, or cyber crimes

---

### ✅ Step-by-Step Process
1. IMMEDIATELY block your bank account/card - Call bank
2. Visit https://cybercrime.gov.in
3. Click 'Report Cybercrime' button
4. Select complaint category (Financial Fraud/Social Media Crime)
5. Fill complete details: transaction ID, amount, date/time
6. Upload ALL evidence: screenshots, emails, chat messages
7. Submit and note down complaint number
8. You will receive acknowledgement on email/SMS
9. If fraud amount > ₹50,000, also visit local cyber police station
10. Follow up after 15 days if no response

---

### ⏳ Timelines
Acknowledgement: 24 hours
Investigation: 15-90 days

---

### 📁 Documents Required
- Transaction screenshots/receipts
- Bank statement (last 3 months)
- Chat/email/SMS screenshots
- ID proof (Aadhaar)
- Address proof

---

### 🚨 Escalation Path
- Local Cyber Cell
- State Cyber Crime Cell
- CBI Cyber Division

-

***LangGraph State***

In [6]:
class AgentState(TypedDict):
    query: str
    language: str
    kb_entry: Optional[Dict[str, Any]]
    response: str

***LangGraph Agents***

In [7]:
def intent_agent(state: AgentState):
    state["kb_entry"] = detect_intent(state["query"], KB)
    return state


def guidance_agent(state: AgentState):
    state["response"] = build_guidance(
        state["kb_entry"],
        state["language"]
    )
    return state

***Building LangGraph Pipeline***

In [8]:
graph = StateGraph(AgentState)

graph.add_node("intent", intent_agent)
graph.add_node("guidance", guidance_agent)

graph.set_entry_point("intent")
graph.add_edge("intent", "guidance")

grievance_graph = graph.compile()

***End To End Testing***

In [9]:
state = {
    "query": "passport verification delay",
    "language": "English"
}

result = grievance_graph.invoke(state)
print(result["response"])


## 🏛 Police Verification Pending / Delay

📄 **Description**  
Police verification not done causing passport delay

---

### ✅ Step-by-Step Process
1. Check passport application status online
2. If status shows 'Police Verification Initiated'
3. Wait for police officer to visit your address
4. They may call you to police station instead
5. If no contact in 15 days, call local police station
6. Provide your file number to SHO
7. Request to expedite verification
8. If police asks for 'fees', this is illegal - report it
9. Call Passport Seva helpline: 1800-258-1800
10. They will coordinate with police
11. Can register complaint if delay > 45 days

---

### ⏳ Timelines
Police Visit: 15-30 days from initiation

---

### 📁 Documents Required
- Passport file number
- Address proof
- All required documents
- Original certificates

---

### 🚨 Escalation Path
- Local Police Station SHO
- SP Office
- Passport Officer
- MEA Grievance

---

### 🔗 Official Links
https://passportindia.gov.in

---

##